In [1]:
import pandas as pd

In [2]:
data_filepath = "gunviolence.pickle"

df = pd.read_pickle(data_filepath)
df

,index,incident_id,date,state,city_or_county,address,n_killed,n_injured,incident_url,source_url,...,sources,state_house_district,state_senate_district,year,suicide,mass shooting,gang,wounded,dead,non-suicide
0,278,95289,2014-01-01,Michigan,Muskegon,300 block of Monroe Avenue,0,0,http://www.gunviolencearchive.org/incident/95289,http://www.mlive.com/news/muskegon/index.ssf/2...,...,http://www.mlive.com/news/muskegon/index.ssf/2...,92.0,34.0,2014,False,False,False,False,False,True
1,279,92401,2014-01-01,New Jersey,Newark,Central Avenue,0,0,http://www.gunviolencearchive.org/incident/92401,http://www.nj.com/essex/index.ssf/2014/01/newa...,...,http://www.nj.com/essex/index.ssf/2014/01/newa...,29.0,29.0,2014,False,False,False,False,False,True
2,280,92383,2014-01-01,New York,Queens,113th Avenue,1,0,http://www.gunviolencearchive.org/incident/92383,http://www.timesledger.com/stories/2014/2/firs...,...,http://www.timesledger.com/stories/2014/2/firs...,33.0,14.0,2014,False,False,False,False,True,True
3,281,92142,2014-01-01,New York,Brooklyn,St. Johns Place,0,1,http://www.gunviolencearchive.org/incident/92142,http://www.nydailynews.com/new-york/nyc-crime/...,...,http://www.nydailynews.com/new-york/nyc-crime/...,43.0,20.0,2014,False,False,False,True,False,True
4,282,95261,2014-01-01,Missouri,Springfield,Beverly Hills and Temple,0,1,http://www.gunviolencearchive.org/incident/95261,http://www.ozarksfirst.com/story/deputies-6-ye...,...,http://www.ozarksfirst.com/story/deputies-6-ye...,131.0,30.0,2014,False,False,False,True,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
236205,239672,1083142,2018-03-31,Louisiana,Rayne,North Riceland Road and Highway 90,0,0,http://www.gunviolencearchive.org/incident/108...,http://www.klfy.com/news/local/rayne-woman-cha...,...,http://www.klfy.com/news/local/rayne-woman-cha...,NaN,NaN,2018,False,False,False,False,False,True
236206,239673,1083139,2018-03-31,Louisiana,Natchitoches,247 Keyser Ave,1,0,http://www.gunviolencearchive.org/incident/108...,http://www.ksla.com/story/37854648/man-wanted-...,...,http://www.ksla.com/story/37854648/man-wanted-...,23.0,31.0,2018,False,False,False,False,True,True
236207,239674,1083151,2018-03-31,Louisiana,Gretna,1300 block of Cook Street,0,1,http://www.gunviolencearchive.org/incident/108...,http://www.nola.com/crime/index.ssf/2018/04/sh...,...,http://www.nola.com/crime/index.ssf/2018/04/sh...,85.0,7.0,2018,False,False,False,True,False,True
236208,239675,1082514,2018-03-31,Texas,Houston,12630 Ashford Point Dr,1,0,http://www.gunviolencearchive.org/incident/108...,https://www.chron.com/news/houston-texas/houst...,...,http://www.khou.com/article/news/hpd-investiga...,149.0,17.0,2018,False,False,False,False,True,True


In [3]:
all_incidents = df[['year', 'state']].groupby(['state', 'year']).size()
all_incidents.name = 'all incidents'

In [4]:
df_incidents = df[['year', 'state', 'suicide', 'mass shooting', 'gang', 'non-suicide']].groupby(['state', 'year']).sum().astype(int)
df_incidents = df_incidents.join(all_incidents)

In [5]:
population_filepath = "nst-est2020.xlsx"
population_df = pd.read_excel(population_filepath, header=3, skiprows=[62, 63, 64, 65, 66, 67])
population_df.rename(columns={'July 1': 2020}, inplace=True)
population_df.dropna(inplace=True)
population_df['state'] = population_df['Unnamed: 0'].str.replace('[^\w\s]','')
population_df = pd.melt(population_df, id_vars=['state'], value_vars=[2014, 2015, 2016, 2017, 2018, 2019, 2020])
population_df.rename(columns={'variable': 'year', 'value':'population'}, inplace=True)
population_df

<ipython-input-5-1a4f61d92135>:5: FutureWarning: The default value of regex will change from True to False in a future version.
  population_df['state'] = population_df['Unnamed: 0'].str.replace('[^\w\s]','')


,state,year,population
0,United States,2014,318386329.0
1,Northeast,2014,56021339.0
2,Midwest,2014,67765576.0
3,South,2014,119666248.0
4,West,2014,74933166.0
...,...,...,...
394,Washington,2020,7693612.0
395,West Virginia,2020,1784787.0
396,Wisconsin,2020,5832655.0
397,Wyoming,2020,582328.0


In [6]:
df_incidents['population'] = list(df_incidents.merge(population_df, on=['year', 'state'], how='left')['population'])
df_incidents

suicide  mass shooting  gang  non-suicide  all incidents  \
state   year                                                             
Alabama 2014       25              2     3         1292           1318   
        2015       29              4     0         1006           1034   
        2016       33             15     3         1276           1309   
        2017       41              7     1         1421           1461   
        2018        8              5     0          339            347   
...               ...            ...   ...          ...            ...   
Wyoming 2014        4              0     0           60             64   
        2015        3              0     0          194            198   
        2016        5              0     0          122            127   
        2017        2              0     0           89             91   
        2018        5              0     0            9             14   

              population  
state   year              
Alabama 2014   4843737.0  
        2015   4854803.0  
        2016   4866824.0  
        2017   4877989.0  
        2018   4891628.0  
...                  ...  
Wyoming 2014    583159.0  
        2015    586389.0  
        2016    585243.0  
        2017    579994.0  
        2018    579054.0  

[250 rows x 6 columns]

In [9]:
df_incidents = df_incidents.div(df_incidents['population'], axis=0).drop(columns=['population'])
df_incidents

AttributeError: 'DataFrame' object has no attribute 'year'

In [8]:
df_incidents.to_pickle('gunViolenceMetadata.pickle')